# SHOE BRAND CAPSTONE PROJECT          

By Subashree Rajkumar


# PHASE1:

Web scrapping the below given six URL :

 

https://www.nike.com

https://www.adidas.com

https://us.puma.com/us/en

https://www.skechers.com

https://www.columbia.com/

https://www.woodlandworldwide.com/


# Installing the tools,Libraries and the webdrivers For WebScrapping

In [27]:
!pip install selenium

In [28]:
!pip install webdriver-manager

In [29]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


from selenium.webdriver.chrome.service import Service

import time

# WebScrapping of 6 URL's.

# 1 Adidas

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json



driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#driver = webdriver.Chrome(service=Service('helpers/chromedriver-win64/chromedriver.exe'))

driver.get("https://www.adidas.co.in/originals-shoes")
soup = BeautifulSoup(driver.page_source, 'html.parser')
element = soup.find_all('a', {'data-auto-id': 'glass-hockeycard-link'})
hrefs = ["https://www.adidas.co.in"+link.get('href') for link in element]
details = []
for index, link in enumerate(hrefs):
    print(f"trying {index} - {link}")
    link_detail = {"link": link, "brand": "adidas"}
    driver.execute_script("window.open('"+link+"', '_blank')")
    window_name = driver.window_handles[-1]
    driver.switch_to.window(window_name=window_name)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    script_tags = soup.find_all('script')
    data_store_value = None
    for script in script_tags:
        script_content = script.string
        if script_content and 'window.DATA_STORE' in script_content:
            data_store_value = script_content.replace('window.DATA_STORE = JSON.parse(', '').rstrip(');')
            break
    json_value = json.loads(json.loads(data_store_value))
    pid = json_value["nuPdp"]["currentProductId"]
    shortlist =  json_value["productStore"]["products"][pid]["data"]
    rating_details = json_value["nuPdp"]["ratingsReviews"]["ratings"]["value"]["right"]
    for element in rating_details["secondaryRatings"]:
        if element["name"] == "Comfort":
            comfort = element["averageRating"]
        elif element["name"] == "Quality":
            quality = element["averageRating"]
        elif element["name"] == "Width":
            width = element["averageRating"]
        
    link_detail["shoe_name"] =  shortlist["name"]
    link_detail["category"] = shortlist["attribute_list"]["gender"]
    link_detail["style_or_product_code"] = shortlist["model_number"]
    link_detail["reviews"] = rating_details["reviewCount"]
    available_sizes = shortlist["attribute_list"]["sizeTypes"]
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = shortlist["attribute_list"]["color"].split(",")
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    link_detail["stars"] = rating_details["overallRating"]
    link_detail["price"] = shortlist["pricing_information"]["currentPrice"]
    link_detail["quality"], link_detail["comfort"], link_detail["width"] = quality, comfort, width   
    details.append(link_detail)
    driver.close()
    window_name = driver.window_handles[0]
    driver.switch_to.window(window_name=window_name)


print(details)
driver.quit()


import pandas as pd
df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "quality", "stars"]
adidas_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in details)
adidas_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in details)
adidas_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in details)
adidas_df3.rename(columns={"quality":"quantification"}, inplace=True)
all_df = pd.DataFrame(details)
all_df.to_csv("adidas_all.csv", index=None)

adidas_df1.to_csv("adidas_1.csv", index=None)
adidas_df2.to_csv("adidas_2.csv", index=None)
adidas_df3.to_csv("adidas_3.csv", index=None)



trying 0 - https://www.adidas.co.in/gazelle-shoes/IG1062.html
trying 1 - https://www.adidas.co.in/gazelle-bold-shoes/ID6990.html
trying 2 - https://www.adidas.co.in/samba-og-shoes/B75807.html
trying 3 - https://www.adidas.co.in/samba-og-shoes/B75806.html
trying 4 - https://www.adidas.co.in/superstar-shoes/EG4959.html
trying 5 - https://www.adidas.co.in/gazelle-shoes/IG4988.html
trying 6 - https://www.adidas.co.in/stan-smith-shoes/Q47225.html
trying 7 - https://www.adidas.co.in/superstar-xlg-shoes/IG9777.html
trying 8 - https://www.adidas.co.in/kick-shoes/IG8951.html
trying 9 - https://www.adidas.co.in/kick-shoes/IG8950.html
trying 10 - https://www.adidas.co.in/falcon-shoes/IG8301.html
trying 11 - https://www.adidas.co.in/adifom-stan-smith-mule-shoes/IG5973.html
trying 12 - https://www.adidas.co.in/falcon-shoes/IG5732.html
trying 13 - https://www.adidas.co.in/superstar-slip-on-shoes/IG5717.html
trying 14 - https://www.adidas.co.in/forum-xlg-shoes/IG2578.html
trying 15 - https://www.adid

# 2 Columbia

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_columbia_details():
    links=set()
    urls = ['https://www.columbia.com/c/fishing-shoes/', 'https://www.columbia.com/c/hiking-boots-shoes/', 'https://www.columbia.com/c/ski-snow-shoes/']
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for i in soup.find_all('a',attrs={'class':"link js-tile-link"}):
            links.add('https://www.columbia.com' + i['href'])
    
    def get_shoe_name(soup):
        return soup.select_one('#main .mb-0').text.strip()

    def get_category(soup):
        category = "Women" if "women" in get_shoe_name(soup).lower() else "Men"
        return category

    def get_style_or_product_code(soup):
        return soup.select_one(".product-id").text.strip()

    def get_reviews(soup):
        script_tags = soup.find_all('script')
        import json
        for script in script_tags:
            script_content = script.string
            if script_content and 'window.digitalData = ' in script_content:
                start_index = script_content.find('window.digitalData = ')
                end_index = script_content.find(' || {};', start_index)
                digital_data = script_content[start_index + len('window.digitalData = '):end_index]
                data = json.loads(digital_data)
                review_count = data["page"]["product"]["productInfo"]["prodReviewCount"]
                star_count = data["page"]["product"]["productInfo"]["productReviewAvg"]
        return review_count

    def get_sizes(soup):
        return [i.text.strip() for i in soup.select('.onhover')]
    
    def get_colors(soup):
        color_elements = soup.select('.grop-product-product a[data-color]')
        return [element['data-color'] for element in color_elements]

    def get_star_count(soup):
        script_tags = soup.find_all('script')
        import json
        for script in script_tags:
            script_content = script.string
            if script_content and 'window.digitalData = ' in script_content:
                start_index = script_content.find('window.digitalData = ')
                end_index = script_content.find(' || {};', start_index)
                digital_data = script_content[start_index + len('window.digitalData = '):end_index]
                data = json.loads(digital_data)
                review_count = data["page"]["product"]["productInfo"]["prodReviewCount"]
                star_count = data["page"]["product"]["productInfo"]["productReviewAvg"]
        return star_count

    def get_price(soup):
        price_div = soup.find('div', class_='price')
        if price_div:
            regular_price_element = price_div.find('span', class_='value')
            regular_price = regular_price_element.get_text(strip=True) if regular_price_element else None
            return regular_price
        return None
    
    details = []
    for index, link in enumerate(links):
        print(f"Trying {index} - {link}")
        link_detail = {"link": link, "brand": "columbia"}
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        link_detail["shoe_name"] =  get_shoe_name(soup)
        link_detail["category"] = get_category(soup)
        link_detail["style_or_product_code"] = get_style_or_product_code(soup)
        link_detail["reviews"] = get_reviews(soup)
        available_sizes = get_sizes(soup)
        link_detail["number_of_sizes"] = len(available_sizes)
        link_detail["size"] = ",".join(available_sizes)
        colors_available = get_colors(soup)
        link_detail["number_of_colors"] = len(colors_available)
        link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
        link_detail["stars"] = get_star_count(soup)
        link_detail["price"] = get_price(soup)
        link_detail["comfort"] = None
        link_detail["quantification"] = None
        details.append(link_detail)
    
    return details

columbia_details = get_columbia_details()
print(columbia_details)


df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "quantification", "stars"]
columbia_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in columbia_details)
columbia_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in columbia_details)
columbia_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in columbia_details)

all_df = pd.DataFrame(columbia_details)
all_df.to_csv("columbia_all.csv", index=None)

columbia_df1.to_csv("columbia_1.csv", index=None)
columbia_df2.to_csv("columbia_2.csv", index=None)
columbia_df3.to_csv("columbia_3.csv", index=None)

Trying 0 - https://www.columbia.com/p/mens-bugaboot-iii-boot-1791221.html?dwvar_1791221_color=010
Trying 1 - https://www.columbia.com/p/mens-pfg-bonehead-shoe-1987131.html?dwvar_1987131_color=088
Trying 2 - https://www.columbia.com/p/womens-moritza-boot-2005041.html?dwvar_2005041_color=286
Trying 3 - https://www.columbia.com/p/womens-firecamp-boot---wide-1800312.html?dwvar_1800312_color=010
Trying 4 - https://www.columbia.com/p/mens-pfg-slack-tide-boat-shoe-2027031.html?dwvar_2027031_color=088
Trying 5 - https://www.columbia.com/p/womens-newton-ridge-bc-boot-2044571.html?dwvar_2044571_color=252
Trying 6 - https://www.columbia.com/p/mens-pfg-bonehead-shoe---wide-1987132.html?dwvar_1987132_color=271
Trying 7 - https://www.columbia.com/p/mens-granite-trail-waterproof-shoe-2053211.html?dwvar_2053211_color=231
Trying 8 - https://www.columbia.com/p/womens-newton-ridge-waterproof-omni-heat-ii-boot-2056181.html?dwvar_2056181_color=089
Trying 9 - https://www.columbia.com/p/mens-pfg-tamiami-shoe

# 3 Nike

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0',
    'Accept': 'image/webp,*/*',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

def individual_shoe_links():
    nike_home_links=['https://www.nike.com/w/retro-running-8kemk','https://www.nike.com/w/womens-shoes-5e1x6zy7ok']
    shoe_links=set()
    for link in nike_home_links:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        product_cards = soup.find_all('div',attrs={'class':'product-card__body'})
        for card in product_cards:
            shoe_links.add(card.find('a')['href'])
    return shoe_links


def get_individual_shoe_details(link):
    def get_shoe_name(soup):
        return soup.find('h1',attrs={"id":'pdp_product_title'}).text.strip()

    def get_category(soup):
        return "Women" if "women" in soup.find('h2').text.strip().lower() else "Men"

    def get_style_or_product_code(soup):
        return soup.select_one('.description-preview__style-color').text.strip()

    def get_reviews(data):
        return data['props']['pageProps']['initialState']['reviews']['total']

    def get_sizes(data):
        sizes = set()
        for _, details in data['props']['pageProps']['initialState']['Threads']['products'].items():
            for sku in details['skus']:
                sizes.add(sku['nikeSize'])
            break
        return sizes

    def get_colors(soup):
        colorway_divs = soup.find_all('div', class_='colorway-container')
        alt_texts = []
        for div in colorway_divs:
            img_tag = div.find('img')
            if img_tag and 'alt' in img_tag.attrs:
                alt_text = img_tag['alt']
                alt_texts.append(alt_text)
        return alt_texts

    def get_star_count(data): 
        return data['props']['pageProps']['initialState']['reviews']['averageRating']

    def get_price(data):
        price = ""
        for _, details in data['props']['pageProps']['initialState']['Threads']['products'].items():
            price = details['fullPrice']
            break
        return price

    def get_dimensions(data):
        fitness = 0
        comfort = 0
        recommend = 0
        for dim in data['props']['pageProps']['initialState']['reviews']["dimensions"]:
            if dim['label'] == "How did this product fit?":
                fitness = dim["average"]
            elif dim['label'] == "How comfortable was this product?":
                comfort = dim["average"]
            elif dim['label'] == "Would you recommend this product?":
                recommend = dim["average"]
        return fitness, comfort, recommend
    link_detail = {"link": link, "brand": "nike"}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    data = json.loads(soup.find('script', id='__NEXT_DATA__').text)
    link_detail["shoe_name"] =  get_shoe_name(soup)
    link_detail["category"] = get_category(soup)
    link_detail["style_or_product_code"] = get_style_or_product_code(soup)
    link_detail["reviews"] = get_reviews(data)
    available_sizes = get_sizes(data)
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = get_colors(soup)
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    link_detail["stars"] = get_star_count(data)
    link_detail["price"] = get_price(data)
    link_detail["fitness"], link_detail["comfort"], link_detail["recommend"] = get_dimensions(data)
    return link_detail

def get_nike_details():
    shoe_links = individual_shoe_links()
    details = []
    for shoe_link in shoe_links:
        details.append(get_individual_shoe_details(shoe_link))
    return details


nike_details = get_nike_details()
print(nike_details)


df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "fitness", "stars"]
nike_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in nike_details)
nike_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in nike_details)
nike_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in nike_details)
nike_df3.rename(columns={"fitness":"quantification"}, inplace=True)
all_df = pd.DataFrame(nike_details)
all_df.to_csv("nike_all.csv", index=None)

nike_df1.to_csv("nike_1.csv", index=None)
nike_df2.to_csv("nike_2.csv", index=None)
nike_df3.to_csv("nike_3.csv", index=None)


[{'link': 'https://www.nike.com/t/waffle-debut-womens-shoes-5Rl1KL/DH9523-105', 'brand': 'nike', 'shoe_name': 'Nike Waffle Debut', 'category': 'Women', 'style_or_product_code': 'Style: DH9523-105', 'reviews': 635, 'number_of_sizes': 15, 'size': '6.5,11.5,8,12,5,9,8.5,6,10,7,10.5,11,5.5,9.5,7.5', 'number_of_colors': 4, 'color_1': 'White/Summit White/White/Blue Tint', 'color_2': 'Black/Orange/Clear/White', 'color_3': 'White/Light Orewood Brown/Sail/Rush Fuchsia', 'color_4': 'White/Black/Orange/White', 'color_5': None, 'stars': 4.7, 'price': 75, 'fitness': 0.75, 'comfort': 1.78, 'recommend': None}, {'link': 'https://www.nike.com/t/cortez-womens-shoes-tZtTd5/DN1791-105', 'brand': 'nike', 'shoe_name': 'Nike Cortez', 'category': 'Women', 'style_or_product_code': 'Style: DN1791-105', 'reviews': 57, 'number_of_sizes': 15, 'size': '6.5,11.5,8,12,5,9,8.5,6,10,7,10.5,11,5.5,9.5,7.5', 'number_of_colors': 6, 'color_1': 'White/Smokey Mauve/Black/Platinum Violet', 'color_2': 'White/Sail/Team Orange/U

# 4 Puma

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0',
    'Accept': 'image/webp,*/*',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

def get_individual_shoe_links():
    home_links = ['https://us.puma.com/us/en/women/shoes?offset=24', 'https://us.puma.com/us/en/men/mens-shoes-and-sneakers?offset=24']
    links=set()
    for link in home_links:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        for i in soup.find_all('li',attrs={'data-test-id':"product-list-item"}):
            link = "https://us.puma.com"+i.find('a')['href']
            link = link.split("?")[0]
            links.add(link)
    return links


def get_individual_shoe_details(link):
    def get_shoe_name(soup): #
        return soup.select_one('#pdp-product-title').text.strip()

    def get_category(soup): # 
        return "Women" if "women" in soup.select_one('#pdp-product-title').text.strip().lower() else "Men"

    def get_style_or_product_code(soup): #
        product_description_div = soup.find('div', {'data-test-id': 'pdp-product-description'})
        info_list = product_description_div.find('ul', {'data-uds-child': 'text'}).find_all('li')
        style = info_list[0].text.split(':')[1].strip()
        return style

    def get_reviews(soup): #
        product_reviews_div = soup.find('div', {'id': 'product-reviews'})
        reviews_heading = product_reviews_div.find('h2')
        review_count = reviews_heading.text.split('(')[-1].split(')')[0].strip()
        return review_count

    def get_sizes(soup): #
        size_picker_div = soup.find('div', {'id': 'size-picker'})
        if not size_picker_div:
            return []
        size_spans = size_picker_div.find_all('label', {'data-size': True})
        return [span.find(class_='text-sm').text for span in size_spans]

    def get_colors(soup): # 
        style_picker_div = soup.find('div', {'id': 'style-picker'})
        img_tags = style_picker_div.find_all('img')
        alt_values = [img.get('alt') for img in img_tags]
        return alt_values

    def get_star_count(soup): #  
        product_reviews_div = soup.find('div', {'id': 'product-reviews'})
        star_count_span = product_reviews_div.find('span', {'class': 'sr-only'})
        star_count = star_count_span.text.split(':')[-1].strip().split('/')[0].strip()
        return star_count

    def get_price(soup): #
        return soup.select_one('.override\:md\:text-2xl').text.strip()

    def get_dimensions(soup): # 
        fitness = 0
        comfort = 0
        recommend = 0  
        return fitness, comfort, recommend
    print(f"trying {link}")
    link_detail = {"link": link, "brand": "puma"}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    link_detail["shoe_name"] =  get_shoe_name(soup)
    link_detail["category"] = get_category(soup)
    link_detail["style_or_product_code"] = get_style_or_product_code(soup)
    link_detail["reviews"] = get_reviews(soup)
    available_sizes = get_sizes(soup)
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = get_colors(soup)
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    link_detail["stars"] = get_star_count(soup)
    link_detail["price"] = get_price(soup)
    link_detail["fitness"], link_detail["comfort"], link_detail["recommend"] = get_dimensions(soup)
    return link_detail
def get_puma_details():
    individual_shoe_links = get_individual_shoe_links()
    details = []
    for shoe_link in individual_shoe_links:
        details.append(get_individual_shoe_details(shoe_link))
    return details

puma_details = get_puma_details()
print(puma_details)


df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "fitness", "stars"]
puma_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in puma_details)
puma_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in puma_details)
puma_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in puma_details)
puma_df3.rename(columns={"fitness":"quantification"}, inplace=True)
all_df = pd.DataFrame(puma_details)
all_df.to_csv("puma_all.csv", index=None)

puma_df1.to_csv("puma_1.csv", index=None)
puma_df2.to_csv("puma_2.csv", index=None)
puma_df3.to_csv("puma_3.csv", index=None)


trying https://us.puma.com/us/en/pd/shibui-cat-slides/385296
trying https://us.puma.com/us/en/pd/rs-x-faded-daytona-sneakers/394981
trying https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332
trying https://us.puma.com/us/en/pd/scoot-zeros-georgia-peach-mens-basketball-shoes/379908
trying https://us.puma.com/us/en/pd/palermo-leather-mens-sneakers/396464
trying https://us.puma.com/us/en/pd/rs-x-soft-womens-sneakers/393772
trying https://us.puma.com/us/en/pd/puma-x-lamelo-ball-rs-x-pocket-lafranc%C3%A9-mens-sneakers/395542
trying https://us.puma.com/us/en/pd/rs-trck-horizon-mens-sneakers/390717
trying https://us.puma.com/us/en/pd/puma-x-lamelo-ball-mb-03-chino-hills-mens-basketball-shoes/379235
trying https://us.puma.com/us/en/pd/fenty-x-puma-avanti-l-womens-sneakers/399262
trying https://us.puma.com/us/en/pd/suede-classic-xxi-womens-sneakers/381410
trying https://us.puma.com/us/en/pd/puma-x-tmc-clyde-og-sneakers/387286
trying https://us.puma.com/us/en/pd/puma-

# 5 Sketchers

In [30]:
pip install undetected_chromedriver

In [31]:

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from bs4 import BeautifulSoup

from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


from selenium.webdriver.chrome.service import Service

individual_urls = []
my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
options = uc.ChromeOptions()
# options.add_argument("--headless")
options.add_argument(f"user-agent={my_user_agent}")


#driver = uc.Chrome(service=Service(ChromeDriverManager().install())), options=options)
driver = uc.Chrome(service=Service('helpers/chromedriver-win64/chromedriver.exe'), options=options)

driver.set_window_size(1920, 1080) 
# Get women shoes
driver.get("https://www.skechers.com/women/shoes/")
soup = BeautifulSoup(driver.page_source, 'html.parser')
div_ = soup.find_all('div', class_="c-product-tile__image-container")
for div in div_:
    individual_urls.append("https://www.skechers.com" + div.find('a')['href'])
# Get men shoes
driver.get("https://www.skechers.com/men/shoes/")
soup = BeautifulSoup(driver.page_source, 'html.parser')
div_ = soup.find_all('div', class_="c-product-tile__image-container")
for div in div_:
    individual_urls.append("https://www.skechers.com" + div.find('a')['href'])
details = []
for index, url in enumerate(individual_urls):
    print(f"trying {index} - {url}")
    link_detail = {"link": url, "brand": "sketchers"}
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    link_detail["shoe_name"] =  soup.select_one('.product-name').text.strip().replace("\n", "").replace("  ","")
    link_detail["category"] = soup.select_one('.c-product-details__label').text.strip().replace("\n", "").replace("  ","")
    link_detail["style_or_product_code"] = soup.select_one('.c-product-details__style').text.strip().replace("\n", "").replace("  ","")
    link_detail["reviews"] = soup.select_one('.pr-snippet-review-count').text.strip().replace("\n", "").replace("  ","").replace('NoReviews',"0")
    available_sizes = [size.text.strip() for size in soup.select('.c-product-attributes__item__value--size')]
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = [color.get("aria-label").strip().replace("Select Color ","") for color in soup.select('.js-color-attr-selector')]
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    star_list = soup.select('.pr-star-v4')
    stars = 0
    for star in star_list:
        count = star.get("class")[-1].split("-")[-2]
        if count.isdigit():
            stars += int(count)/100
    link_detail["stars"] = stars
    link_detail["price"] = soup.select_one('.sales .value').text.strip()
    link_detail["quality"], link_detail["comfort"], link_detail["width"] = 0,0,0
    details.append(link_detail)

driver.quit()

import pandas as pd
df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "quality", "stars"]
sketchers_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in details)
sketchers_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in details)
sketchers_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in details)
sketchers_df3.rename(columns={"quality":"quantification"}, inplace=True)
all_df = pd.DataFrame(details)
all_df.to_csv("sketchers_all.csv",index=False)

sketchers_df1.to_csv("sketchers_1.csv",index=False)
sketchers_df2.to_csv("sketchers_2.csv",index=False)
sketchers_df3.to_csv("sketchers_3.csv",index=False)

#  6 Woodlandworldwide

In [15]:
import requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0',
    'Accept': 'image/webp,*/*',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

links = set()
def get_product_details(product):
    link = product["skus"][0]["Link"]
    if link in links:
        return
    links.add(link)
    link_detail = {"link": link, "brand": "woodlandworldwide"}
    link_detail["shoe_name"] =  product["skus"][0]["name"]
    link_detail["category"] = product["gender"]
    link_detail["style_or_product_code"] = product["productID"]
    link_detail["reviews"] = "0"
    available_sizes = [sku["color"] for sku in product["skus"]]
    link_detail["number_of_sizes"] = len(available_sizes)
    link_detail["size"] = ",".join(available_sizes)
    colors_available = [size["size"]  for sku in product['skus'] for size in sku["sizes"]]
    link_detail["number_of_colors"] = len(colors_available)
    link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
    link_detail["stars"] = 0
    link_detail["price"] = product["price"]
    link_detail["fitness"], link_detail["comfort"], link_detail["recommend"] = 0, 0, 0
    return link_detail


response = requests.get("https://2iupd8w418.execute-api.ap-south-1.amazonaws.com/product/category/WOMEN_SNEAKERS", headers= headers).json()
details = []
for product in response["products"]:
    detail = get_product_details(product)
    if detail:
        details.append(detail)

requests.get("https://2iupd8w418.execute-api.ap-south-1.amazonaws.com/product/category/MEN_SNEAKERS", headers= headers).json()
for product in response["products"]:
    detail = get_product_details(product)
    if detail:
        details.append(detail)

import pandas as pd
df1_columns = ["link","shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["link","number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["link","reviews", "size", "comfort", "fitness", "stars"]
www_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in details)
www_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in details)
www_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in details)
www_df3.rename(columns={"fitness":"quantification"}, inplace=True)
all_df = pd.DataFrame(details)
all_df.to_csv("www_all.csv", index=None)

www_df1.to_csv("www_1.csv", index=None)
www_df2.to_csv("www_2.csv", index=None)
www_df3.to_csv("www_3.csv", index=None)